In [1]:
import shutil
import os
from glob import glob
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import yaml
import shutil

In [2]:
EXTENSION_CSV = ".csv"
EXTENSION_JSON = ".json"
EXTENSION_JPG = ".jpg"
EXTENSION_TXT = ".txt"

✅ YOLO 데이터셋으로 등록할 원본 데이터셋의 경로를 '**PATH_DATASET**'에 입력합니다.

In [3]:
PATH_DATASET = "/content/drive/MyDrive/canAiry/data/02_2nd-dataset/01-2_EP-motor"

# 원본 데이터셋 확인

In [5]:
PATH_CSV = PATH_DATASET + "/00_csv/"
PATH_CSV_TRAIN = PATH_CSV + "train/"
PATH_CSV_VAL = PATH_CSV + "val/"
PATH_CSV_TEST = PATH_CSV + "test/"
PATH_JSON = PATH_DATASET + "/01_label/"
PATH_JSON_TRAIN = PATH_JSON + "train/"
PATH_JSON_VAL = PATH_JSON + "val/"
PATH_JSON_TEST = PATH_JSON + "test/"
PATH_TLC = PATH_DATASET + "/02_tlc/"
PATH_TLC_TRAIN = PATH_TLC + "train/"
PATH_TLC_VAL = PATH_TLC + "val/"
PATH_TLC_TEST = PATH_TLC + "test/"

> 샘플 수를 확인합니다.

In [6]:
csv_train = glob(PATH_CSV_TRAIN+'*'+EXTENSION_CSV)
csv_val = glob(PATH_CSV_VAL+'*'+EXTENSION_CSV)
csv_test = glob(PATH_CSV_TEST+'*'+EXTENSION_CSV)
csv_size = [len(csv_train), len(csv_val), len(csv_test)]
csv_size

[960, 120, 120]

In [7]:
json_train = glob(PATH_JSON_TRAIN+'*'+EXTENSION_JSON)
json_val = glob(PATH_JSON_VAL+'*'+EXTENSION_JSON)
json_test = glob(PATH_JSON_TEST+'*'+EXTENSION_JSON)
json_size = [len(json_train), len(json_val), len(json_test)]
json_size

[960, 120, 120]

In [8]:
tlc_train = glob(PATH_TLC_TRAIN+'*'+EXTENSION_JPG)
tlc_val = glob(PATH_TLC_VAL+'*'+EXTENSION_JPG)
tlc_test = glob(PATH_TLC_TEST+'*'+EXTENSION_JPG)
tlc_size = [len(tlc_train), len(tlc_val), len(tlc_test)]
tlc_size

[0, 0, 0]

🟥 이미지와 라벨 간 서로 쌍(Pair)이 맞는 지 확인합니다. 🟥

In [9]:
csv_name_train = [os.path.splitext(csv.split('/')[-1])[0] for csv in csv_train]
json_name_train = [os.path.splitext(label.split('/')[-1])[0] for label in json_train]
tlc_name_train = [os.path.splitext(tlc.split('/')[-1])[0] for tlc in tlc_train]
paired_name_train = list(set(csv_name_train) & set(json_name_train) & set(tlc_name_train))
len(paired_name_train)

0

In [10]:
csv_name_val = [os.path.splitext(csv.split('/')[-1])[0] for csv in csv_val]
json_name_val = [os.path.splitext(label.split('/')[-1])[0] for label in json_val]
tlc_name_val = [os.path.splitext(tlc.split('/')[-1])[0] for tlc in tlc_val]
paired_name_val = list(set(csv_name_val) & set(json_name_val) & set(tlc_name_val))
len(paired_name_val)

0

In [11]:
csv_name_test = [os.path.splitext(csv.split('/')[-1])[0] for csv in csv_test]
json_name_test = [os.path.splitext(label.split('/')[-1])[0] for label in json_test]
tlc_name_test = [os.path.splitext(tlc.split('/')[-1])[0] for tlc in tlc_test]
paired_name_test = list(set(csv_name_test) & set(json_name_test) & set(tlc_name_test))
len(paired_name_test)

0

In [12]:
# def make_sample_pair(sample_unpaired, sample_name_paired):
#   for sample in sample_unpaired:
#     sample_name = os.path.splitext(sample.split('/')[-1])[0]
#     if sample_name not in sample_name_paired:
#       if os.path.isfile(sample):
#           os.remove(sample)

# YOLO 데이터셋 구축

✅ 클래스별 **인덱스와 이름**이 중복되지 않도록 설정합니다.

In [13]:
CLASS_ID = 2
CLASS_NAME = 'EP-motor'

## 경로 설정

In [14]:
PATH_DATASET_YOLO = "/content/drive/MyDrive/object_detection/datasets/canAIry/"+CLASS_NAME
if os.path.isdir(PATH_DATASET_YOLO) == False:
  os.mkdir(PATH_DATASET_YOLO)

In [15]:
PATH_TRAIN = PATH_DATASET_YOLO + "/train/"
if os.path.isdir(PATH_TRAIN) == False:
  os.mkdir(PATH_TRAIN)

PATH_TRAIN_IMAGE = PATH_TRAIN + "images/"
if os.path.isdir(PATH_TRAIN_IMAGE) == False:
  os.mkdir(PATH_TRAIN_IMAGE)

PATH_TRAIN_LABEL = PATH_TRAIN + "labels/"
if os.path.isdir(PATH_TRAIN_LABEL) == False:
  os.mkdir(PATH_TRAIN_LABEL)

In [16]:
PATH_VAL = PATH_DATASET_YOLO + "/val/"
if os.path.isdir(PATH_VAL) == False:
  os.mkdir(PATH_VAL)

PATH_VAL_IMAGE = PATH_VAL + "images/"
if os.path.isdir(PATH_VAL_IMAGE) == False:
  os.mkdir(PATH_VAL_IMAGE)

PATH_VAL_LABEL = PATH_VAL + "labels/"
if os.path.isdir(PATH_VAL_LABEL) == False:
  os.mkdir(PATH_VAL_LABEL)

In [17]:
PATH_TEST = PATH_DATASET_YOLO + "/test/"
if os.path.isdir(PATH_TEST) == False:
  os.mkdir(PATH_TEST)

PATH_TEST_IMAGE = PATH_TEST + "images/"
if os.path.isdir(PATH_TEST_IMAGE) == False:
  os.mkdir(PATH_TEST_IMAGE)

PATH_TEST_LABEL = PATH_TEST + "labels/"
if os.path.isdir(PATH_TEST_LABEL) == False:
  os.mkdir(PATH_TEST_LABEL)

## 이미지 데이터셋 구축

### 방법1. Raw 이미지(.jpg) 생성

In [18]:
def check_csv(df_temp):
  if df_temp.shape[0] != 512:
    return False
  list_temp = [row[1][0].split(';')[:-1] for row in df_temp.iterrows()]
  for temp in list_temp:
    if len(temp) != 640:
      return False
  return True

In [19]:
def create_raw_image(list_csv, path_dest):
  for path_csv in tqdm(list_csv):
    arr_image = np.zeros((512, 640))
    df_temp = pd.read_csv(path_csv)[4:].reset_index(drop=True)
    sample_name = os.path.splitext(path_csv.split('/')[-1])[0]
    # csv 파일 내 온도 데이터가 나타내는 픽셀의 형상(640x512)을 확인합니다.
    if check_csv(df_temp):
      for idx, row in df_temp.iterrows():
        arr_image[idx, :] = np.array([float(x) for x in row[0].split(';')[:-1]])
      # Raw 이미지(.jpg) 파일을 생성합니다.
      path_jpg = path_dest+sample_name+EXTENSION_JPG
      if os.path.exists(path_jpg) == False:
        plt.imsave(path_jpg, arr_image)
    else:
      if os.path.exists(path_csv):
        os.remove(path_csv)
  return

In [20]:
create_raw_image(csv_train, PATH_TRAIN_IMAGE)
create_raw_image(csv_val, PATH_VAL_IMAGE)
create_raw_image(csv_test, PATH_TEST_IMAGE)

100%|██████████| 120/120 [01:01<00:00,  1.94it/s]


### 방법2. 기본 열화상 이미지 사용

In [ ]:
def copy_dataset(list_sample, path_dest):
  for sample in tqdm(list_sample):
    sample_name = os.path.splitext(sample.split('/')[-1])[0]
    extension = os.path.splitext(sample.split('/')[-1])[1]
    shutil.copy(sample, path_dest+sample_name+extension)

In [ ]:
copy_dataset(tlc_train, PATH_TRAIN_IMAGE)
copy_dataset(tlc_val, PATH_VAL_IMAGE)
copy_dataset(tlc_test, PATH_TEST_IMAGE)

100%|██████████| 120/120 [00:00<00:00, 354.16it/s]


## 라벨(.txt) 생성

In [21]:
def coco_to_yolo(bbox, img_size):
    return [((2*bbox[0] + bbox[2])/(2*img_size[0])) , ((2*bbox[1] + bbox[3])/(2*img_size[1])), bbox[2]/img_size[0], bbox[3]/img_size[1]]

In [22]:
def make_label_yolo(list_json, path_dest):
  for path_json in tqdm(list_json):
    sample_name = os.path.splitext(path_json.split('/')[-1])[0]
    with open(path_json) as r:
      dict_label = json.load(r)
      img_size = [dict_label["metadata"]["width"], dict_label["metadata"]["height"]]
      for obj in dict_label["annotations"]:
        bbox = list(obj["data"].values())
        X, Y, W, H = coco_to_yolo(bbox, img_size)
        label = f"{CLASS_ID} {X} {Y} {W} {H}\n"
        if obj["attributes"]["status"] != "normal":
          label = f"{CLASS_ID+1} {X} {Y} {W} {H}\n"
        # YOLO 라벨(.txt)을 생성합니다.
        path_txt = path_dest+sample_name+EXTENSION_TXT
        if os.path.exists(path_txt) == False:
          with open(path_txt, "a") as w:
            w.write(label)

In [23]:
make_label_yolo(json_train, PATH_TRAIN_LABEL)
make_label_yolo(json_val, PATH_VAL_LABEL)
make_label_yolo(json_test, PATH_TEST_LABEL)

100%|██████████| 120/120 [00:33<00:00,  3.64it/s]
